In [6]:
import os
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import load_model
import mediapipe as mp

# Load model and label encoder
model = load_model('asl_cnn_2D_model.h5')

with open("label_encoder.pickle", "rb") as f:
    label_encoder = pickle.load(f)

CLASS_NAMES = list(label_encoder.classes_)

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

# Image directory
IMAGE_FOLDER_PATH = r"ASL_Alphabet_Dataset\Amir Dataset"

# Storage
true_labels = []
predicted_labels = []

for image_name in os.listdir(IMAGE_FOLDER_PATH):
    image_path = os.path.join(IMAGE_FOLDER_PATH, image_name)

    if not image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # Extract true label
    filename = os.path.splitext(image_name)[0]
    if "_" not in filename:
        print(f"Skipping file (no underscore): {image_name}")
        continue

    raw_label = filename.split('_')[0]
    true_label = ''.join([c for c in raw_label if not c.isdigit()])

    if true_label not in CLASS_NAMES:
        print(f"Skipping unknown class: {true_label} in {image_name}")
        continue

    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for point in hand_landmarks.landmark:
                landmarks.append(point.x)
                landmarks.append(point.y)

            if len(landmarks) < 42:
                print(f"Incomplete landmarks in {image_name}")
                continue

            input_data = np.array(landmarks[:42]).reshape(1, -1)
            prediction = model.predict(input_data)
            predicted_index = np.argmax(prediction, axis=1)[0]
            predicted_label = CLASS_NAMES[predicted_index]

            print(f"✅ {image_name} — True: {true_label}, Predicted: {predicted_label}")

            true_labels.append(true_label)
            predicted_labels.append(predicted_label)

# Accuracy & Confusion Matrix
if true_labels and predicted_labels:
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f"\n✅ Model Accuracy: {accuracy:.4f}")

    labels_sorted = sorted(set(true_labels + predicted_labels))
    cm = confusion_matrix(true_labels, predicted_labels, labels=labels_sorted)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels_sorted, yticklabels=labels_sorted)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix - ASL MLP Model (Amir)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print("\n📋 Classification Report:")
    print(classification_report(true_labels, predicted_labels, labels=CLASS_NAMES))
else:
    print("❌ No valid predictions were made.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
✅ A1_test1.jpg — True: A, Predicted: A
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
✅ A2_test.jpg — True: A, Predicted: A
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
✅ A_test.jpg — True: A, Predicted: A
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
✅ B1_test.jpg — True: B, Predicted: B
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
✅ B2_test.jpg — True: B, Predicted: B
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
✅ B_test.jpg — True: B, Predicted: B
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
✅ C1_test.jpg — True: C, Predicted: C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
✅ C2_test.jpg — True: C, Predicted: C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
✅ C_test.jpg — True: C, Predicted: C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
✅ D1_test.jpg — True: D, Predicted: I
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ D2_test.jpg — True: D, Predicted: I
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
✅ D_test.jpg — True: D, Predicted: V
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
✅ E1_test.jpg — True: E, Predicted: S
1/1 ━━━━━━━━━━━


📋 Classification Report:
              precision    recall  f1-score   support

           A       0.33      1.00      0.50         3
           B       0.60      1.00      0.75         3
           C       1.00      1.00      1.00         3
           D       0.00      0.00      0.00         3
           E       0.00      0.00      0.00         3
           F       1.00      0.33      0.50         3
           G       0.67      0.67      0.67         3
           H       0.75      1.00      0.86         3
           I       0.25      0.33      0.29         3
           J       0.00      0.00      0.00         3
           K       0.40      0.67      0.50         3
           L       0.43      1.00      0.60         3
           M       0.50      0.33      0.40         3
           N       0.00      0.00      0.00         3
           O       0.00      0.00      0.00         3
           P       0.00      0.00      0.00         1
           Q       0.25      0.50      0.33         2
 

C:\Users\shree\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shree\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shree\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is